# Part 4 V2- Cluster Metro Areas
In this notebook, I will use Agglomerative​ clustering to cluster the location data.  


In [1]:
#import libraries
import requests
import numpy as np
import pandas as pd
import ibm_boto3
import io
from sklearn.cluster import AgglomerativeClustering
from datetime import datetime

In [2]:
# import plotting libraries
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
#Import the first venue dataframe
obj = resource.Object(bucket_name=bucket, key='VenueData.json').get()
df = pd.read_json(io.BytesIO(obj['Body'].read()))
df.sort_index(inplace=True)
df.head()

,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10
0,Tokyo,Japan,Asia,35.682839,139.759455,Café,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Lounge,Garden,French Restaurant,Dessert Shop,Thai Restaurant,Steakhouse
1,Delhi,India,Asia,28.651718,77.221939,Indian Restaurant,Bar,Café,Flea Market,Lounge,Asian Restaurant,Ice Cream Shop,South Indian Restaurant,Portuguese Restaurant,Plaza
2,Mexico City,Mexico,North America,19.432601,-99.133342,Mexican Restaurant,Art Museum,Museum,Arts & Crafts Store,Bar,Ice Cream Shop,Boutique,Bakery,Restaurant,Café
3,São Paulo,Brazil,South America,-23.550651,-46.633382,Brazilian Restaurant,Japanese Restaurant,Café,Bakery,Snack Place,Bookstore,Cosmetics Shop,Asian Restaurant,Art Gallery,Chinese Restaurant
4,Lagos,Nigeria,Africa,6.455057,3.394179,Lounge,African Restaurant,Bar,Café,Shopping Mall,Pizza Place,Hotel,Art Gallery,Italian Restaurant,Market


In [5]:
#Import the venue dataframe used for clustering
obj = resource.Object(bucket_name=bucket, key='VenueOneHot.json').get()
df_onehot = pd.read_json(io.BytesIO(obj['Body'].read()))
df_onehot.head()

,Metro,ATM,Acai House,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Amphitheater,Antique Shop,...,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Yoshoku Restaurant,Zhejiang Restaurant,Zoo Exhibit
0,Ahmedabad,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,Alexandria,0.0,0.0,0.0,0.0,0.0,0.0,0.010309,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
10,Berlin/Brandenburg,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
11,Bogotá,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
12,Boston,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,0.0,0.0,...,0.0,0.0,0.01,0.0,0.0,0.0,0.02,0.0,0.0,0.0


In [23]:
l = len(df_onehot.index)
for i in range(5,26):
    model = AgglomerativeClustering(n_clusters=i).fit(df_onehot.drop('Metro',axis=1))
    aLabels = model.labels_
    v = pd.DataFrame({'cat':aLabels,'size':range(l)})
    print('\n i=', i,'\n',v.groupby('cat', as_index=False).count().sort_values('size', ascending=False))


 i= 5 
    cat  size
0    0    69
2    2    15
4    4     8
1    1     7
3    3     1

 i= 6 
    cat  size
0    0    67
2    2    15
4    4     8
1    1     7
5    5     2
3    3     1

 i= 7 
    cat  size
5    5    66
0    0    15
4    4     8
1    1     7
2    2     2
3    3     1
6    6     1

 i= 8 
    cat  size
5    5    66
3    3    13
4    4     8
1    1     7
0    0     2
2    2     2
6    6     1
7    7     1

 i= 9 
    cat  size
2    2    66
3    3    13
4    4     8
1    1     7
0    0     2
5    5     1
6    6     1
7    7     1
8    8     1

 i= 10 
    cat  size
0    0    66
3    3    13
4    4     8
1    1     7
2    2     1
5    5     1
6    6     1
7    7     1
8    8     1
9    9     1

 i= 11 
     cat  size
1     1    62
3     3    13
10   10     8
0     0     7
4     4     4
2     2     1
5     5     1
6     6     1
7     7     1
8     8     1
9     9     1

 i= 12 
     cat  size
1     1    62
3     3    13
10   10     8
0     0     6
4     4     4
2     2   

n=14 seems to be the most interesting balance between number of clusters and cluster size

In [24]:
#Add cluster labels
n = 14
model = AgglomerativeClustering(n_clusters=n).fit(df_onehot.drop('Metro',axis=1))
aLabels = model.labels_
df_onehot.insert(0, 'Cluster Labels', aLabels)
df_onehot.head()

,Cluster Labels,Metro,ATM,Acai House,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Amphitheater,...,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Yoshoku Restaurant,Zhejiang Restaurant,Zoo Exhibit
0,10,Ahmedabad,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,4,Alexandria,0.0,0.0,0.0,0.0,0.0,0.0,0.010309,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
10,1,Berlin/Brandenburg,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
11,2,Bogotá,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
12,2,Boston,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,0.0,...,0.0,0.0,0.01,0.0,0.0,0.0,0.02,0.0,0.0,0.0


In [25]:
#Add cluster labels
dftemp = df_onehot.iloc[:,[0,1]]
df_New = df.join(dftemp.set_index('Metro'), on='Metropolitan')
df_New.head()

,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
0,Tokyo,Japan,Asia,35.682839,139.759455,Café,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Lounge,Garden,French Restaurant,Dessert Shop,Thai Restaurant,Steakhouse,1
1,Delhi,India,Asia,28.651718,77.221939,Indian Restaurant,Bar,Café,Flea Market,Lounge,Asian Restaurant,Ice Cream Shop,South Indian Restaurant,Portuguese Restaurant,Plaza,10
2,Mexico City,Mexico,North America,19.432601,-99.133342,Mexican Restaurant,Art Museum,Museum,Arts & Crafts Store,Bar,Ice Cream Shop,Boutique,Bakery,Restaurant,Café,2
3,São Paulo,Brazil,South America,-23.550651,-46.633382,Brazilian Restaurant,Japanese Restaurant,Café,Bakery,Snack Place,Bookstore,Cosmetics Shop,Asian Restaurant,Art Gallery,Chinese Restaurant,1
4,Lagos,Nigeria,Africa,6.455057,3.394179,Lounge,African Restaurant,Bar,Café,Shopping Mall,Pizza Place,Hotel,Art Gallery,Italian Restaurant,Market,1


In [26]:
df_New.loc[:,['Metropolitan','Country','Cluster Labels']].head(15)

,Metropolitan,Country,Cluster Labels
0,Tokyo,Japan,1
1,Delhi,India,10
2,Mexico City,Mexico,2
3,São Paulo,Brazil,1
4,Lagos,Nigeria,1
5,Mumbai,India,10
6,Cairo,Egypt,2
7,Kyoto,Japan,1
8,Wuhan,China,3
9,Los Angeles,United States,2


In [27]:
#Save dataframe to json file
resource.Bucket(name=bucket).put_object(Key='VenueDataWithAGGClusters.json', Body=df_New.to_json())
print('Saved')

Saved


In [28]:
#make a backup of the file
t = datetime.now().strftime('%Y%m%d_%H%M')
resource.Bucket(name=bucket).put_object(Key='VenueDataWithAGGClusters' + t + '.json', Body=df_New.to_json())
print('Saved ' + t)

Saved 20190915_2009
